In [ ]:
from brian2 import *
prefs.codegen.target = "numpy"
import numpy as np
import scipy.special as sp_spec

In [ ]:
def bin_array(array, BIN, time_array):
    """
    Bin Brian data for plotting
    """
    N0 = int(BIN/(time_array[1]-time_array[0]))
    N1 = int((time_array[-1]-time_array[0])/BIN)
    return array[:N0*N1].reshape((N1,N0)).mean(axis=1)

In [ ]:
def Brian_run(prbC, nu_ext, excitatory_params, inhibitory_params, TotTime, timeStep, T, percentage):
    start_scope()
    #time parameters
    DT=timeStep*10**3; TotTime=TotTime*10**3
    
    defaultclock.dt = DT*ms; duration = TotTime*ms
    BIN = T*10**3; time_array = arange(int(TotTime/DT))*DT
    
    N1, Qi, Tvi, Eei, Eii, Ii, gizi, Eizi, ai, bi, ci, di, Tui, Tsyni = inhibitory_params
    N2, Qe, Tve, Eee, Eie, Ie, gize, Eize, ae, be, ce, de, Tue, Tsyne = excitatory_params
    eqs="""
    dv/dt=(g_iz*(v-E_iz)**2-u-GsynE*(v-Ee)-GsynI*(v-Ei)-I)/Tv:1 (unless refractory)
    du/dt=(a*(b*v-u))/Tu:1
    dGsynI/dt = -GsynI/Tsyn : 1
    dGsynE/dt = -GsynE/Tsyn : 1  
    a:1
    b:1
    c:1
    d:1
    g_iz:1
    E_iz:1
    Tsyn:second
    Tv:second
    Tu:second
    Ee:1
    Ei:1
    I:1
    """

    # Population 1 - Fast Spiking
    G_inh = NeuronGroup(N1, eqs, threshold='v > -30.', reset='v = c; u=u+d',refractory='5*ms', method='heun')
    #Initial conditions
    G_inh.v=-40; G_inh.u=0
    #Parameters
    G_inh.a=ai; G_inh.b=bi; G_inh.c=ci; G_inh.d=di
    G_inh.Tsyn=Tsyni*second; G_inh.Tv=Tvi*second; G_inh.Tu=Tui*second
    G_inh.Ee=Eei; G_inh.Ei=Eii; G_inh.I=Ii
    G_inh.g_iz=gizi; G_inh.E_iz=Eizi

In [ ]:
    # Population 2 - Regular Spiking
    G_exc = NeuronGroup(N2, eqs, threshold='v > -30.', reset='v = c; u=u+d',refractory='5*ms', method='heun')
    #Initial conditions
    G_exc.v=-40; G_exc.u=1.
    #Parameters
    G_exc.a=ae; G_exc.b=be; G_exc.c=ce; G_exc.d=de
    G_exc.Tsyn=Tsyne*second; G_exc.Tv=Tve*second; G_exc.Tu=Tue*second
    G_exc.Ee=Eee; G_exc.Ei=Eie; G_exc.I=Ie
    G_exc.g_iz=gize; G_exc.E_iz=Eize

    # external drive--------------------------------------------------------------------------
    P_ed=PoissonGroup(N2, rates=nu_ext*Hz)

    # 1 is inhibitory, 2 is excitatory generally here
    S_12 = Synapses(G_inh, G_exc, on_pre='GsynI_post+=Qi') # give the 2 populations to be connected, and the increment on the postsynaptic neuron when there is a spike on the presynaptic neuron
    S_12.connect('i!=j', p=prbC)
    # don't use _pre and _post in variable names in brian
    
    S_11 = Synapses(G_inh, G_inh, on_pre='GsynI_post+=Qi')
    S_11.connect('i!=j',p=prbC)
    
    S_21 = Synapses(G_exc, G_inh, on_pre='GsynE_post+=Qe')
    S_21.connect('i!=j',p=prbC)
    
    S_22 = Synapses(G_exc, G_exc, on_pre='GsynE_post+=Qe')
    S_22.connect('i!=j', p=prbC)
    
    S_ed_in = Synapses(P_ed, G_inh, on_pre='GsynE_post+=Qe') # P_ed is for the external drive, since it's excitatory we can say that we add Qe for both neurons exc and inh receiving input from ext
    S_ed_in.connect(p=prbC)
    
    S_ed_ex = Synapses(P_ed, G_exc, on_pre='GsynE_post+=Qe')
    S_ed_ex.connect(p=prbC)
    
    ### RECORDING TRICK #########################
    PgroupE = NeuronGroup(1, 'P:1', method='heun')
    PE=Synapses(G_exc, PgroupE, 'P_post = v_pre : 1 (summed)')
    PE.connect(p=1)
    
    PgroupI = NeuronGroup(1, 'P:1', method='heun')
    PI=Synapses(G_inh, PgroupI, 'P_post = v_pre : 1 (summed)')
    PI.connect(p=1)
    
    P2mon = StateMonitor(PgroupE, 'P', record=0)
    P1mon = StateMonitor(PgroupI, 'P', record=0)
    
    ############################################
    
    #M1G_inh = SpikeMonitor(G_inh); M1G_exc = SpikeMonitor(G_exc)
    FRG_inh = PopulationRateMonitor(G_inh) # pop firing rate recording
    FRG_exc = PopulationRateMonitor(G_exc)

    # Run simulation -------------------------------------------------------------------------------
    #print('--##Start simulation##--')
    run(duration)
    #print('--##End simulation##--')
    #time data
    TimBinned = bin_array(time_array, BIN, time_array)
    #get total population potential
    Pe = bin_array(P2mon[0].P, BIN, time_array)
    Pi = bin_array(P1mon[0].P, BIN, time_array)
    #mean population potential across time
    Pi = Pi/N1; Pe = Pe/N2
    #take its mean steady state value
    mean_pot_I = np.mean(Pi[int(percentage*len(Pi))::])
    mean_pot_E = np.mean(Pe[int(percentage*len(Pe))::])
    
    #get mean firing rates
    LfrG_exc = array(FRG_exc.rate/Hz)
    popRateG_exc = bin_array(LfrG_exc, BIN, time_array)
    LfrG_inh = array(FRG_inh.rate/Hz)
    popRateG_inh = bin_array(LfrG_inh, BIN, time_array)
    
    mean_firing_rate_E = np.mean(popRateG_exc[int(percentage*len(popRateG_exc))::])
    mean_firing_rate_I = np.mean(popRateG_inh[int(percentage*len(popRateG_inh))::])
    
    trajectories = [TimBinned/1000, Pe, Pi, popRateG_exc, popRateG_inh]
    mean_results = [mean_pot_E, mean_pot_I, mean_firing_rate_E, mean_firing_rate_I]
    
    return trajectories, mean_results